In [5]:
import run
import numpy as np
from ac.utils.utils import abs_path
import ac.data.db as db
import pickle
import os.path
from tqdm import tqdm

In [2]:
# load relation indices
path_relations_ind = abs_path("data/relations_ind.npy")
relation_ind = np.load(path_relations_ind)
print("Relation ind shape", relation_ind.shape)

# load id array
path_ids = abs_path("data/ids.npy")
ev_ids = np.load(path_ids)
print("Eventuality ids shape", ev_ids.shape)

Relation ind shape (10256017, 3)
Eventuality ids shape (8146286, 1)


In [3]:
path_aserdb = abs_path("data/aser_v0.1.0.db")
conn = db.open_db_connection(path_aserdb)
c = conn.cursor()

In [4]:
query_cache = {}
def get_eventuality_text_cached(c, id_str):
    if id_str in query_cache:
        return query_cache[id_str]
    text = db.get_eventuality_text(c, id_str)
    query_cache[id_str] = text
    return text

In [6]:
# populate cache, requires about 8GB ram
load_words_query = "SELECT _id, words FROM Eventualities;"
for (id_str, words) in c.execute(load_words_query):
    query_cache[id_str] = words

In [8]:
dataset = []
dirpath_datachunks = abs_path("data/chunks/")
chunk_id = 0

# save dataset to pickle, requires about (another!) 7GB of ram
for i in tqdm(range(relation_ind.shape[0])):
    id_str1 = ev_ids[relation_ind[i,0]][0]
    id_str2 = ev_ids[relation_ind[i,1]][0]
    relation_type = relation_ind[i,2]
    text1 = get_eventuality_text_cached(c, id_str1)
    text2 = get_eventuality_text_cached(c, id_str2)
    dataset.append((text1, relation_type, text2))

chunkfile = os.path.join(dirpath_datachunks, "dataset.pickle")
with open(chunkfile, 'wb') as f:
    pickle.dump(dataset, f)
dataset = []

100%|██████████| 10256017/10256017 [00:35<00:00, 290516.92it/s]
